[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-diffusion-diffusers-colab/blob/main/flax_stable_diffusion_3.ipynb)

In [ ]:
!pip install -q jax==0.3.25 jaxlib==0.3.25 flax==0.6.2 transformers accelerate
!pip install -q git+https://github.com/huggingface/diffusers

import jax, random, gc
from flax.jax_utils import replicate
from flax.training.common_utils import shard
from diffusers import FlaxStableDiffusionPipeline, FlaxStableDiffusionXLPipeline, StableDiffusionXLPipeline
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu('tpu_driver_nightly')
jax.device_count()
import numpy as np
from PIL import Image

pipeline, params = FlaxStableDiffusionPipeline.from_pretrained('uf/dark-sushi-25d', from_pt=True, dtype=jax.numpy.bfloat16, safety_checker=None)
params = replicate(params)

In [ ]:
gc.collect()

prompt = "cute duck"
negative_prompt = "blurry, ugly"

num_samples = jax.device_count()

prompt_n = num_samples * [prompt]
prompt_ids = pipeline.prepare_inputs(prompt_n)
prompt_ids = shard(prompt_ids)

negative_prompt_n = num_samples * [negative_prompt]
negative_prompt_ids = pipeline.prepare_inputs(negative_prompt_n)
negative_prompt_ids = shard(negative_prompt_ids)

real_seed = random.randint(0, 2147483647)
prng_seed = jax.random.PRNGKey(real_seed)
prng_seed = jax.random.split(prng_seed, jax.device_count())

images = pipeline(prompt_ids, params, prng_seed, neg_prompt_ids=negative_prompt_ids, num_inference_steps=50, height=512, width=512, guidance_scale=7.5, jit=True).images
images = pipeline.numpy_to_pil(np.asarray(images.reshape((num_samples,) + images.shape[-3:])))
images[0]
images[1]
images[2]
images[3]
images[4]
images[5]
images[6]
images[7]